In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os, sys, h5py
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sb
import tensorflow as tf
from deepomics import neuralnetwork as nn
from deepomics import utils, fit, visualize, saliency

import mutagenesis_functions as mf
import time as time

In [2]:
# load hairpin data
data_path = '../data_peter/random_sequences_with_custom_hl.hdf5'
with h5py.File(data_path) as dataset:
    X_train_seq = np.expand_dims(np.array(dataset['train_seq']).transpose([0, 2, 1]), axis=2)
    X_train_struct = np.expand_dims(np.array(dataset['train_structures']).transpose([0, 2, 1]), axis=2)
    Y_train = np.expand_dims(np.array(dataset['train_targets']), axis=1)
    X_test_seq = np.expand_dims(np.array(dataset['test_seq']).transpose([0, 2, 1]), axis=2)
    X_test_struct = np.expand_dims(np.array(dataset['test_structures']).transpose([0, 2, 1]), axis=2)
    Y_test = np.expand_dims(np.array(dataset['test_targets']), axis=1)

num_data, seq_length, _, num_alphabet = X_train_seq.shape

# get validation set from training set
valid_frac = 0.2
N = len(X_train_seq)
split_index = int(N*valid_frac)
shuffle = np.random.permutation(N)

# put in a dictionary for deepomics
train = {'inputs': X_train_seq[shuffle[split_index:]], 
         'targets': Y_train[shuffle[split_index:]]}
valid = {'inputs': X_train_seq[shuffle[:split_index]], 
         'targets': Y_train[shuffle[:split_index]]}
test = {'inputs': X_test_seq, 'targets': Y_test}



In [3]:
def cnn_model(input_shape, output_shape):

    # create model
    layer1 = {'layer': 'input', #41
            'input_shape': input_shape
            }
    
    layer2 = {'layer': 'dense',        # input, conv1d, dense, conv1d_residual, dense_residual, conv1d_transpose,
                                        # concat, embedding, variational_normal, variational_softmax, + more
              'num_units': 196,
              'norm': 'batch',          # if removed, automatically adds bias instead
              'activation': 'relu',     # or leaky_relu, prelu, sigmoid, tanh, etc
              'dropout': 0.5,           # if removed, default is no dropout
             }
    
    layer3 = {'layer': 'dense',
            'num_units': output_shape[1],
            'activation': 'sigmoid'
            }

    model_layers = [layer1, layer2, layer3]

    # optimization parameters
    optimization = {"objective": "binary",
                  "optimizer": "adam",
                  "learning_rate": 0.0003,
                  "l2": 1e-5,
                  #"label_smoothing": 0.05,
                  #"l1": 1e-6,
                  }
    return model_layers, optimization

tf.reset_default_graph()

# get shapes of inputs and targets
input_shape = list(train['inputs'].shape)
input_shape[0] = None
output_shape = train['targets'].shape

# load model parameters
model_layers, optimization = cnn_model(input_shape, output_shape)

# build neural network class
nnmodel = nn.NeuralNet(seed=247)
nnmodel.build_layers(model_layers, optimization)

# compile neural trainer
save_path = '../results/toyhairpin'
param_path = os.path.join(save_path, 'fullyconnected_1_196')
nntrainer = nn.NeuralTrainer(nnmodel, save='best', file_path=param_path)

In [4]:
# look at the shapes of each layer
nnmodel.inspect_layers()

----------------------------------------------------------------------------
Network architecture:
----------------------------------------------------------------------------
layer1: inputs
(?, 41, 1, 4)
layer2: dense_0
(?, 196)
layer3: dense_0_batch
(?, 196)
layer4: dense_0_active
(?, 196)
layer5: dense_0_dropout
(?, 196)
layer6: dense_1
(?, 1)
layer7: dense_1_bias
(?, 1)
layer8: output
(?, 1)
----------------------------------------------------------------------------


In [5]:
# initialize session
sess = utils.initialize_session()

# set best parameters
nntrainer.set_best_parameters(sess)

# test model
loss, mean_vals, std_vals = nntrainer.test_model(sess, test, name='test')

loading model from:  ../results/toyhairpin/fullyconnected_1_196_best.ckpt
  test  loss:		0.00255
  test  accuracy:	0.99998+/-0.00000
  test  auc-roc:	1.00000+/-0.00000
  test  auc-pr:		1.00000+/-0.00000


In [6]:
#Get Predictions

# calculate activations
#predictionsoutput = nntrainer.get_activations(sess, test, layer='output')
#predictionslogits = nntrainer.get_activations(sess, test, layer='dense_0_bias')

In [7]:
import mutagenesis_functions as mf


In [8]:
# initialize session
sess = utils.initialize_session()

# set best parameters
nntrainer.set_best_parameters(sess)



num_summary = 1000
X = test['inputs'][:num_summary]

sum_mut2_scores = mf.som_average(X, nntrainer, sess)


loading model from:  ../results/toyhairpin/fullyconnected_1_196_best.ckpt
0
Epoch duration =0hr 0min 0.2s
Cumulative duration =0hr 0min 0.2s

1
Epoch duration =0hr 0min 0.13s
Cumulative duration =0hr 0min 0.34s

2
Epoch duration =0hr 0min 0.2s
Cumulative duration =0hr 0min 0.54s

3
Epoch duration =0hr 0min 0.15s
Cumulative duration =0hr 0min 0.69s

4
Epoch duration =0hr 0min 0.17s
Cumulative duration =0hr 0min 0.86s

5
Epoch duration =0hr 0min 0.19s
Cumulative duration =0hr 0min 1.05s

6
Epoch duration =0hr 0min 0.14s
Cumulative duration =0hr 0min 1.19s

7
Epoch duration =0hr 0min 0.14s
Cumulative duration =0hr 0min 1.33s

8
Epoch duration =0hr 0min 0.14s
Cumulative duration =0hr 0min 1.47s

9
Epoch duration =0hr 0min 0.14s
Cumulative duration =0hr 0min 1.61s

10
Epoch duration =0hr 0min 0.15s
Cumulative duration =0hr 0min 1.75s

11
Epoch duration =0hr 0min 0.13s
Cumulative duration =0hr 0min 1.88s

12
Epoch duration =0hr 0min 0.14s
Cumulative duration =0hr 0min 2.03s

13
Epoch duratio

KeyboardInterrupt: 

In [ ]:
X[:1].shape

In [35]:


def sectotime(t):
    t = np.around(t, 2)
    if t>=3600.:
        s = t%60
        m = ((t)%3600)//60
        hr = t//3600
        output = str(int(hr)) + 'hr ' + str(int(m)) + 'min ' + str(s) + 's'
    else:
        if t>=60.:
            s = t%60
            m = t//60
            output = str(int(m)) + 'min ' + str(s) + 's'
        else:
            s = np.copy(t)
            output = str(s) + 's'
    return(output)
    


In [38]:
t = 129.3697

sectotime(t)

'2min 9.37s'